In [1]:
import pennylane as qml
from pennylane import numpy as np
import torch
import sympy
import cirq
import random

In [2]:
def create_layer(qubits, layer_id):
    symbols = [sympy.Symbol(layer_id + '-' + str(i)) for i in range(len(qubits))]
    
    return symbols

In [3]:
n_qubits = 6
data_qubits = cirq.GridQubit.rect(1, n_qubits)
data_qubits

[cirq.GridQubit(0, 0),
 cirq.GridQubit(0, 1),
 cirq.GridQubit(0, 2),
 cirq.GridQubit(0, 3),
 cirq.GridQubit(0, 4),
 cirq.GridQubit(0, 5)]

### Phase I

In [4]:
def create_layer(qubits, layer_id):
    symbols = [sympy.Symbol(layer_id + '-' + str(i)) for i in range(len(qubits))]
    gate_set = [cirq.rx, cirq.ry, cirq.rz]

    gates = []
    symbol_gates = []
    for i, q in enumerate(qubits):
        chosen_gate = random.choice(gate_set)
        gates.append(chosen_gate(0)(q))
        symbol_gates.append(chosen_gate(symbols[i])(q))

    for control, target in zip(qubits, qubits[1:]):
        gates.append(cirq.CZ(control, target))
        symbol_gates.append(cirq.CZ(control, target))

    return gates, symbol_gates, symbols

In [5]:
dev = qml.device('default.qubit', wires = 6)

In [6]:
n_qubits = 6
@qml.qnode(dev)
def layer(params: np.array):
    gate_set = [qml.RX, qml.RY, qml.RZ]
    
    for i in range(n_qubits):
        chosen_gate = random.choice(gate_set)
        chosen_gate(params[i], wires = i)
        
    tuples = [(i,i+1) for i in range(n_qubits-1)]

    for tup in tuples:
        qml.CZ(wires=[tup[0], tup[1]])
        
    return qml.expval(qml.PauliZ(1))